In [47]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import os 
import sklearn
from sklearn.model_selection import train_test_split


def Normlize(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmean) / (Zmax - Zmin)
    return Z

def Normlize2(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmin) / (Zmax - Zmin)
    return Z


def Data_Reading(Normalization=True):
    trainset = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\nos-data\\3times7class\\trainset2.npy')
    trainlabel = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\nos-data\\3times7class\\trainlabel2.npy')
    testset = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\nos-data\\3times7class\\testset2.npy')
    testlabel = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\nos-data\\3times7class\\testlabel2.npy')

    # Normalization
    trainset = Normlize(trainset)
    testset = Normlize(testset)

    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(trainset).type(torch.cuda.FloatTensor)
        train_y = torch.from_numpy(trainlabel).type(torch.int64)
        test_x = torch.from_numpy(testset).type(torch.cuda.FloatTensor)
        test_y = torch.from_numpy(testlabel).type(torch.int64)
    else:
        data = torch.from_numpy(data).type(torch.FloatTensor)
        label = torch.from_numpy(label).type(torch.int64)

    # reshape
    train_x = train_x.view(525, 10, 1, 120)
    train_x2 = train_x.view(525, 1, 10, 120)
    test_x = test_x.view(175, 10, 1, 120)
    test_x2 = test_x.view(175, 1, 10, 120)
    
    # data = data.cpu().numpy()
    # label = label.numpy()    
    # train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.3) 
    # train_x = torch.from_numpy(train_x).type(torch.cuda.FloatTensor)
    # test_x = torch.from_numpy(test_x).type(torch.cuda.FloatTensor)
    # train_y = torch.from_numpy(train_y).type(torch.int64)
    # test_y = torch.from_numpy(test_y).type(torch.int64)
    

    permutation = np.random.permutation(train_y.shape[0])
    train_x = train_x[permutation, :, :, :]
    print('---------------------------------------------------------------------------')
    train_y = train_y[permutation]
    return train_x, test_x, train_y, test_y


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1,3,(1,3),stride=(1,1))#（10-1）/1+1 = 10 （120 - 3）/1 + 1 = 118
        # self.conv2 = nn.Conv2d(3,6,(10,1),stride=(1,1))#1   118
        # self.conv3 = nn.Conv2d(6,10,(1,4),stride=(1,2))
        # self.fc1 = nn.Linear(10*1*58,7)
        self.conv1 = nn.Conv2d(10,5,(1,3),stride=(1,1))#10 :（120 - 3）/1 + 1 = 118  
        self.conv2 = nn.Conv2d(5,10,(1,3),stride=(1,1))#1 : (118-3)/1+1 = 116
        self.conv3 = nn.Conv2d(10,15,(1,3),stride=(1,1))#1 : (116-3)/1+1= 114
        self.fc1 = nn.Linear(15*1*114,427)#427
        self.fc2 = nn.Linear(427,7)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x#


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#cuda:0

    print(device)

    cnn = Net()
    print(cnn)
    cnn.to(device)

    #sgd -> stochastic gradient descent
    lrr = 0.01
    optimizer = optim.SGD(cnn.parameters(), lr=lrr, momentum=0.8)#
    loss_func = nn.CrossEntropyLoss()#CrossEntropyLoss()

    train_x, test_x, train_y, test_y = Data_Reading(Normalization=1)
    train_y = train_y.squeeze()
    test_y = test_y.squeeze()
    train_x = train_x.to(device)
    test_x = test_x.to(device)
    train_y = train_y.to(device)
    test_y = test_y.to(device)

    

    #train
    sum = 0
    max = 0
    
    batch_size = 21
    tr_x = Variable(train_x)
    tr_y = Variable(train_y)
    for epoch in range(150):
        running_loss = 0.0
        for i in range(0,(int)(len(train_x)/batch_size)):
            t_x = Variable(train_x[i*batch_size:i*batch_size+batch_size])
            t_y = Variable(train_y[i*batch_size:i*batch_size+batch_size])
            out = cnn(t_x)
            loss = loss_func(out, t_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
   
        running_loss = running_loss / 25

        out = cnn(tr_x)
        predicted_train = torch.max(out.data, 1)[1]
        total_train = tr_y.size(0)
        for j in range(tr_y.size(0)):
            if predicted_train[j] == tr_y[j]:
                sum += 1
        
        print('total_train:{}, accuracy:{}, sum:{}'.format(total_train, sum / total_train, sum))
        sum = 0

        if (sum / total_train > 0.90) :
            optimizer = optim.SGD(cnn.parameters(), lr=lrr/10, momentum=0.8/4)
        elif (sum / total_train > 0.95) :
            optimizer = optim.SGD(cnn.parameters(), lr=lrr/10/10, momentum=0)#momentum=0


        print('Epoch[{}], loss: {:.8f}'.format(epoch + 1, running_loss))
    
    
    #test
        te_x = Variable(test_x)
        te_y = Variable(test_y)
        out1 = cnn(te_x)
        predicted_test = torch.max(out1.data, 1)[1]#.data.squeeze()
        total = te_y.size(0)

        for j in range(te_y.size(0)):
            if predicted_test[j] == te_y[j]:
                sum += 1
        
        if(max < sum/total):
            max = sum/total
            maxepoch = epoch + 1
        print('total:{}, accuracy:{}, sum:{}, max={}, maxepoch={}'.format(total, sum / total, sum, max, maxepoch))
        print('=============================================================================')
        sum = 0

total_train:525, accuracy:0.8857142857142857, sum:465
Epoch[69], loss: 0.34059293
total:175, accuracy:0.64, sum:112, max=0.6914285714285714, maxepoch=59
total_train:525, accuracy:0.8895238095238095, sum:467
Epoch[70], loss: 0.33156929
total:175, accuracy:0.6342857142857142, sum:111, max=0.6914285714285714, maxepoch=59
total_train:525, accuracy:0.88, sum:462
Epoch[71], loss: 0.32692434
total:175, accuracy:0.6628571428571428, sum:116, max=0.6914285714285714, maxepoch=59
total_train:525, accuracy:0.8819047619047619, sum:463
Epoch[72], loss: 0.32012535
total:175, accuracy:0.6514285714285715, sum:114, max=0.6914285714285714, maxepoch=59
total_train:525, accuracy:0.8990476190476191, sum:472
Epoch[73], loss: 0.30880069
total:175, accuracy:0.6514285714285715, sum:114, max=0.6914285714285714, maxepoch=59
total_train:525, accuracy:0.8971428571428571, sum:471
Epoch[74], loss: 0.28740732
total:175, accuracy:0.6285714285714286, sum:110, max=0.6914285714285714, maxepoch=59
total_train:525, accuracy:

10.714285714285714


In [13]:
import random
index = random.sample(range(0,10),10)
print(index)

[0, 2, 4, 7, 3, 1, 9, 8, 6, 5]


In [29]:
torch.save(cnn.state_dict(), '\\net\\net_parameters.pkl')

In [45]:
for m in cnn.modules():
    params = list(m.named_parameters())
    print(params)

[('conv1.weight', Parameter containing:
tensor([[[[-0.3761,  0.1001, -0.4186, -0.1839]],

         [[-0.2533, -0.4018, -0.4310, -0.5048]],

         [[-0.2632, -0.3185,  0.1013, -0.0373]],

         [[-0.4070, -0.1267, -0.3912, -0.4335]],

         [[ 0.0283,  0.0544, -0.2606, -0.4131]],

         [[ 0.1456,  0.0818,  0.0535, -0.2685]],

         [[-0.2558, -0.1103,  0.2508,  0.1695]],

         [[ 0.0783,  0.1154,  0.0144, -0.0385]],

         [[ 0.4976,  0.2357,  0.4006,  0.4427]],

         [[ 0.2270, -0.0259, -0.1455, -0.0803]]],


        [[[-0.4097, -0.2975, -0.1239, -0.0020]],

         [[-0.1164, -0.2830,  0.0770, -0.1116]],

         [[ 0.1473, -0.4038, -0.2259, -0.3002]],

         [[-0.0664, -0.2939, -0.2580, -0.0312]],

         [[-0.2715,  0.0019, -0.0976, -0.1689]],

         [[-0.2187, -0.1226,  0.1654,  0.1052]],

         [[-0.3563,  0.1089,  0.1260, -0.1577]],

         [[ 0.2518, -0.2247,  0.2262, -0.1033]],

         [[ 0.1777,  0.5143,  0.2523,  0.4627]],

        